## Network Link-to-Link attribute Validation Checks

Functions used to identify Link-to-Link pairs and compare values. This ncludes:
- Single Attribute In-Link / Out-Link checks
- Multiple Attribute In-Link / Out-Link checks

#### Import Libraries 

In [1]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# from nxviz import CircosPlot
from shapely.geometry import LineString

### Load Network data

#### Load Network using 'IN_HIGHWAY': 
Only load links that are identified as part of the Highway Model Network

In [2]:
use_in_hwy = True

#### Create Function: *segment_heading*: 
Calculate Heading for both Initial segment and Final segment of Link

In [3]:
def segment_heading(line, first_segment=True, flip=False):
    if flip:
        line = LineString(line.coords[::-1])
    
    segment = line.coords[:2] if first_segment else line.coords[-2:]
    
    from_pt = segment[0]
    to_pt = segment[1]
    
    y_delta = to_pt[1] - from_pt[1]
    x_delta = to_pt[0] - from_pt[0]

    angle_radians = np.arctan2(y_delta, x_delta)
    if angle_radians < 0:
        angle_radians = 2 * np.pi + angle_radians;

    # Deal with the angles being from East Counterclockwise to Northerly bearings
    degrees = 90 - angle_radians * 180 / np.pi
    if degrees < 0:
        degrees = degrees + 360

    return degrees

#### Load Node and Line Network data from Shapefiles
Use GeoPandas to load Geospatial data  
Call function 'segment_heading' to calculate Heading

In [4]:
line_shp = gpd.read_file(os.path.join('..', '..', 'data', 'networks', 'client_provided', 'roadway', 'base_2014', 'Chatt_Master.shp')).to_crs(epsg=2274)
node_shp = gpd.read_file(os.path.join('..', '..', 'data', 'networks', 'client_provided', 'roadway', 'base_2014', 'Chatt_Master_Node.shp')).to_crs(epsg=2274)

if use_in_hwy: line_shp = line_shp[line_shp['IN_HIGHWAY']==1].copy()

# Heading AB
line_shp['AB_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True)
line_shp['AB_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False)
# Heading BA
line_shp['BA_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True, flip=True)
line_shp['BA_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False, flip=True)

#### Define groupings

In [5]:
# FUNCCLASS FHWA Functional Classification Code
funcclass_dict = {
    1:'Rural Interstate',
    2:'Rural Principal Arterial',
    6:'Rural Minor Arterial',
    7:'Rural Major Collector',
    8:'Rural Minor Collector',
    9:'Rural Local',
    11:'Urban Interstate',
    12:'Urban Freeway',
    14:'Other Principal Arterial',
    16:'Urban Minor Arterial',
    17:'Urban Collector',
    19:'Urban Local',
    91:'1-Lane Roundabout',
    92:'2-Lane Roundabout',
    98:'Centroid Connector',
    99:'Unmodeled'
}

    
interstate_fc_k = [1, 11, 12]
interstate_fc_v = [funcclass_dict[c] for c in interstate_fc_k]

interstate_df = pd.DataFrame(funcclass_dict.items(), columns=['CLASS_ID', 'CLASS_NAME'], dtype=str)

### Build Network Graph

#### Turn Two-Way Links into 2 One-Way Links for Directional Flow.
#### Create Function: *directional_links*
Keep AB Links and produce BA Links, carrying over link attributes based on the direction of flow.

In [6]:
def directional_links(atts, line_shp):
    dirs = {'keep':'AB', 'invert':'BA'}
    dfs = {}
    
    for flow,d in dirs.items():
        links = line_shp[line_shp['{}_LANES'.format(d)] > 0].copy()
        rename_dict = {'{}{}'.format(d, a):a for a in atts}
        links = links.rename(columns=rename_dict)
        
        if flow == 'invert':
            links['A'] = links['TO_ID']
            links['B'] = links['FROM_ID']
            links = links.drop(columns=['FROM_ID', 'TO_ID']).rename(columns={'A': 'FROM_ID', 'B': 'TO_ID'})
        
        dfs['{}_links'.format(d)] = links
    
    links = pd.concat(dfs.values(), ignore_index=True)
    
    drop_dict = ['{}{}'.format(d, a) for a in atts for d in dirs.values()]
    links = links.drop(columns=drop_dict)
    
    return links

#### Generate Directional links with Attributes
####  Call Function: *directional_links*
Use list of Directional Attributes to carry over, e.g. 'AB_LANES' and 'BA_LANES', to '_LANES' 

In [7]:
# List of Directional Attributes
atts = [
    '_LINKDIR',
    '_LINKDTR',
    '_LANES',
    '_PARKING',
    '_TRIMS',
    '_AADT',
    '_CAR_ADT',
    '_SUT_ADT',
    '_MUT_ADT',
    '_BASEVOL',
    '_AFFTIME',
    '_AFFSPD',
    '_UCDELAY',
    '_DLYCAP',
    '_AMCAP',
    '_PMCAP',
    '_BPRA',
    '_BPRB',
    '_START_HEADING',
    '_END_HEADING'
]
# Generate Directional Links
line_shp = directional_links(atts, line_shp)

####  Build the Directional Graph: *g*
#### Call Method: *nx.from_pandas_edgelist*

In [8]:
g = nx.from_pandas_edgelist(
    line_shp,
    source='FROM_ID',
    target='TO_ID',
    edge_attr=True,
    create_using=nx.DiGraph()
)

assert nx.is_directed(g)

### Link to Link validation Functions

#### Create Function: *link_matcher_atts*: 
For each node in Network, match all combinations of In/ Out Links and return both In and Out values

In [9]:
def link_matcher_atts(attributes, graph, edge_id='ID'):
    heading = True if '_HEADING' in attributes else False
    atts = []
    atts.extend([a for a in attributes if a!='_HEADING'])

    node_feed = []
    node_not_in_network = 0

    for node in node_shp['ID']:
        if node not in g:
#             print('Node not in network: {}'.format(node))
            node_not_in_network += 1
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]
                values = [node, *[d[edge_id] for d in data], *[d[a] for a in atts for d in data]]
                if heading: values.extend([i_data['_END_HEADING'], o_data['_START_HEADING']])
                node_feed.append(values)
                
    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node', *[ d+'_'+edge_id for d in dirs], *[ d+'_'+a for a in atts for d in dirs]]
    if heading: cols.extend([ d+'__HEADING' for d in dirs])
    
#     print('Nodes Not in Network: {:,}'.format(node_not_in_network))
    return pd.DataFrame(node_feed, columns=cols)

#### Create Function: *attribute_change*: 
Calculate change in Attribute

In [10]:
def attribute_change(network, attributes):
    for att in attributes:
        in_fld = 'in_{}'.format(att)
        out_fld = 'out_{}'.format(att)
        fld_diff = 'diff_{}'.format(att)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(network[in_fld][0], (int, float)):
            network[fld_diff] = network[out_fld] - network[in_fld]
        elif isinstance(network[in_fld][0], str):
            network[fld_diff] = 'SAME' if network[out_fld].str == network[in_fld].str else 'CHANGES'

    return network#.groupby([in_fld, out_fld]).count().reset_index()

#### Create Function: *attribute_filtering*: 
Apply Attributes and Values difference to use as filters

In [11]:
def attribute_filtering(network, att_filters):
    for filter_name, filter_val in att_filters.items():
        in_fld = 'in_{}'.format(filter_name)
        out_fld = 'out_{}'.format(filter_name)
        fld_diff = 'diff_{}'.format(filter_name)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(filter_val, (int, float)):
            network = network[abs(network[fld_diff]) >= filter_val]
        elif isinstance(filter_val, str):
            network = network[(network[in_fld] == filter_val) | (network[out_fld] == filter_val)]

    return network#.groupby([in_fld, out_fld]).count().reset_index()

#### Create Function: *export_geo*: 
Export Geometry to Shapefile

In [12]:
# In/Out Long Table
def export_geo(network, export_name, att_name):
    fld_diff = 'diff_{}'.format(att_name)
    
    network = network.rename(columns={'out_ID':'ID'})

    # geometries
    geo_cols = ['ID', 'FROM_ID', 'geometry']
    network = pd.merge(line_shp[geo_cols], network, left_on=['ID', 'FROM_ID'], right_on=['ID', 'node'])
    network.drop(columns=['FROM_ID'], inplace=True)
    
    shp_export = os.path.join('..', '..', 'data', '{}_{}.shp'.format(export_name, '_'.join([n for n in att_name])))
    network.to_file(shp_export)
#     return network#.sort_values(by=fld_diff, ascending=False), shp_export
    return network, shp_export

#### Create Function: *summary_in_out*: 
Create function to aggregate In-link and Out-Link by Attribute and summarize by count

In [13]:
def summary_in_out(network, att, csv_name, filter_in=None, filter_out=None, return_summary = True, include_same_att=False):
    in_att = 'in_{}'.format(*att)
    out_att = 'out_{}'.format(*att)
    
    if include_same_att == False:
        network = network[network[in_att]!=network[out_att]]
        
    if filter_in is not None: network = network[network[in_att].isin(filter_in)]
    if filter_out is not None: network = network[network[out_att].isin(filter_out)]
        
    if return_summary == True:
        network = network.groupby([in_att, out_att])[in_att].count().reset_index(name='count')
    
    csv_export = os.path.join('..', '..', 'data', '{}_{}_SUMMARY.csv'.format(csv_name, *attribute_name))
    network.to_csv(csv_export, index=False)
    print('Table exported to {}'.format(csv_export))

    return network

#### Create Function: *compare_attributes*: 
Create function to call Link-to-Link attribute comparison

In [14]:
def compare_attributes(attribute_names, attribute_filters, export_geometry, export_name, heading_filter=None):
    network = link_matcher_atts(attribute_names, graph=g)
    network = attribute_change(network, attribute_names)
    network = attribute_filtering(network,
                                attribute_filters,
                                )
    try:
        if isinstance(heading_filter, (int, float)): network = network[abs(network['diff__HEADING'])<=heading_filter]
    except KeyError:
        print('Warning: HEADING values not found, heading_filter not applied')

    csv_export = os.path.join('..', '..', 'data', '{}_{}.csv'.format(export_name, '_'.join([n for n in attribute_names])))
    network.to_csv(csv_export, index=False)
    
    # Print
    print_txt = 'Total network links with change in {}: {:,}'
    print(print_txt.format(', '.join([n for n in attribute_names]),
                           len(network)
                           ))
    print('Table exported to {}'.format(csv_export))
    if export_geometry:
        network, shp_export = export_geo(network, export_name, attribute_filters)
        print('Shapefile exported to {}'.format(shp_export))

    return network.drop(columns=['geometry'])

#### Create Function: *compare_attribute_single*: 
Create function to call Link-to-Link for single attribute comparison

In [15]:
### RUN FUNCTIONS
def compare_attribute_single(att_name, export_name, in_att_filters, out_att_filters, return_summary, include_same_att):
    network = link_matcher_atts(att_name, graph=g)

    network = summary_in_out(network,
                             att_name,
                             export_name,
                             in_att_filters,
                             out_att_filters,
                             return_summary,
                             include_same_att
                             )
    
    print_txt = 'Total records for {}: {:,}'
    print(print_txt.format(*att_name, len(network)))
    
    if return_summary == False:
        network, shp_export = export_geo(network, export_name, att_name)
        print('Shapefile exported to {}'.format(shp_export))
        
    return network#.astype(int)

## Run Network VALIDATION TESTS
Use Heading ('_HEADING') to calculate deviation of In-Link(Final Segment) and Out-Link (Initial Segment)


### Link Length
Compare Length Coded value and Length Calculated value (from Geometry)

Projected CRS: EPSG:2274  
Name: NAD83 / Tennessee (ftUS)  
Area of Use: USA - Tennessee
Coordinate Operation:  
 -name: SPCS83 Tennessee zone (US Survey feet)  
 -method: Lambert Conic Conformal (2SP)  

In [16]:
line_shp_len = line_shp[['ID', 'LENGTH', 'geometry']].copy()

line_shp_len['LENGTH_GEO'] = line_shp_len['geometry'].length / 5280
line_shp_len['LENGTH_DIFF'] = (line_shp_len['LENGTH_GEO'] - line_shp_len['LENGTH'])  / line_shp_len['LENGTH']
line_shp_len[['LENGTH', 'LENGTH_GEO', 'LENGTH_DIFF']].describe()

,LENGTH,LENGTH_GEO,LENGTH_DIFF
count,24888.000000,24888.000000,24888.000000
mean,0.176633,0.176641,0.000043
std,0.244548,0.244555,0.000267
min,0.000751,0.000751,-0.035932
25%,0.045927,0.045929,0.000028
50%,0.089672,0.089676,0.000046
75%,0.202572,0.202579,0.000062
max,3.976941,3.976860,0.006728


### Functional Class Coding
Functional Class Code and Functional Class Name comparison validation: FUNCCLASS and FUNC_CLASS
Join FUNCCLASS to FHWA Functional Classification Code

In [17]:
check_columns = ['FUNCCLASS', 'FUNC_CLASS']
export_name = 'FunctionalClass_coding'

### RUN CHECK
report = line_shp[check_columns].copy()
report['FUNCCLASS'] =  pd.to_numeric(report['FUNCCLASS'], errors='coerce').astype('Int64').astype(str)

report = report.fillna('EMPTY')
report = report.groupby(check_columns)['FUNCCLASS'].count().reset_index(name='count')



report = pd.merge(report, interstate_df, left_on='FUNCCLASS', right_on='CLASS_ID', how='left')

csv_export = os.path.join('..', '..', 'data', '{}_{}.csv'.format(export_name, '_'.join([n for n in check_columns])))
report.to_csv(csv_export, index=False)
print('Table exported to {}'.format(csv_export))

report

Table exported to ..\..\data\FunctionalClass_coding_FUNCCLASS_FUNC_CLASS.csv


,FUNCCLASS,FUNC_CLASS,count,CLASS_ID,CLASS_NAME
0,-98,EMPTY,4,NaN,NaN
1,1,Rural Interstate,80,1,Rural Interstate
2,11,EMPTY,6,11,Urban Interstate
3,11,Urban Interstate,693,11,Urban Interstate
4,12,Other Freeways and Expressways,447,12,Urban Freeway
5,14,Urban Principal Arterial,2587,14,Other Principal Arterial
6,16,EMPTY,4,16,Urban Minor Arterial
7,16,Urban Minor Arterial,7626,16,Urban Minor Arterial
8,17,Urban Major Collector,3750,17,Urban Collector
9,18,Urban Minor Collector,895,NaN,NaN


### Functional Class Linkage
Functional Class IN to Functional Class OUT comparison validation: FUNCCLASS

In [18]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['FUNCCLASS']
# Filter by OUT attribute value (Equal to):
in_attribute_filters = None
out_attribute_filters = None
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = True
#Export name:
export_name = 'FunctionalClass_linkage'


### RUN FUNCTIONS
network = compare_attribute_single(attribute_name,
                              export_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

### GENERATE MATRIX
network = network.dropna().pivot(index='in_FUNCCLASS', columns='out_FUNCCLASS', values='count')
csv_export = os.path.join('..', '..', 'data', '{}_{}_MATRIX.csv'.format(export_name, *attribute_name))
network.to_csv(csv_export, index=False)
print('Matrix exported to {}'.format(csv_export))
    
network.fillna('')

Table exported to ..\..\data\FunctionalClass_linkage_FUNCCLASS_SUMMARY.csv
Total records for FUNCCLASS: 194
Matrix exported to ..\..\data\FunctionalClass_linkage_FUNCCLASS_MATRIX.csv


out_FUNCCLASS,-98.0,1.0,2.0,6.0,7.0,8.0,9.0,11.0,12.0,14.0,16.0,17.0,18.0,19.0,91.0,92.0,98.0,99.0
in_FUNCCLASS,,,,,,,,,,,,,,,,,,
-98.0,2,,,,,,,,,,2,,,2,,,4,
1.0,,75,3,1,3,,,4,,,,1,,,,,2,
2.0,,4,299,8,12,12,20,1,1,2,,2,,1,,,35,2
6.0,,,6,252,6,11,16,,,,20,1,,1,,,45,
7.0,,4,10,6,526,9,70,,,,1,17,2,5,,,86,
8.0,,,12,11,9,606,60,,,,,2,5,,,,100,
9.0,,,20,16,70,60,812,,,3,2,7,4,38,,,182,1
11.0,,5,1,,,,,727,4,43,57,9,,8,,,2,4
12.0,,,1,,,,,5,464,47,41,7,6,7,,,1,1


### Zone Connectors to Interstate
Node Link-IN  Link-OUT Summary validation: FUNCTIONAL CLASS

In [19]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['FUNCCLASS']
# Filter by OUT attribute value (Equal to):
in_attribute_filters = [98]
out_attribute_filters = [1,11,12]
# Return Summary table or Link level table
return_summary = False
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export name:
export_name = 'Network_attribute_change'


### RUN FUNCTIONS
network = compare_attribute_single(attribute_name,
                              export_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

network

Table exported to ..\..\data\Network_attribute_change_FUNCCLASS_SUMMARY.csv
Total records for FUNCCLASS: 4
Shapefile exported to ..\..\data\Network_attribute_change_FUNCCLASS.shp


,ID,geometry,node,in_ID,in_FUNCCLASS,out_FUNCCLASS
0,17366,"LINESTRING (2126398.079 222864.885, 2126436.84...",16469,38569,98.0,1.0
1,17481,"LINESTRING (2258663.499 195248.681, 2258566.52...",16608,38599,98.0,1.0
2,17809,"LINESTRING (2127635.228 237908.558, 2127341.82...",16941,38571,98.0,11.0
3,730,"LINESTRING (2257180.305 290252.921, 2257356.37...",2750,38589,98.0,11.0


### Right Shoulder Width Summary
Node Link-IN  Link-OUT Summary validation: RIGHT SHOULDER WIDTH

In [20]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['RS_WIDTH']
# Filter by IN / OUT attribute value (Equal to):
in_attribute_filters = None
out_attribute_filters = [12]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export name:
export_name = 'Right_Shoulder_Width_Summary'


### RUN FUNCTIONS
network = compare_attribute_single(attribute_name,
                              export_name,
                              in_attribute_filters,
                              out_attribute_filters,
                              return_summary,
                              include_same_att
                              )

network

Table exported to ..\..\data\Right_Shoulder_Width_Summary_RS_WIDTH_SUMMARY.csv
Total records for RS_WIDTH: 12


,in_RS_WIDTH,out_RS_WIDTH,count
0,0.0,12.0,29
1,1.0,12.0,20
2,2.0,12.0,32
3,3.0,12.0,1
4,4.0,12.0,6
5,6.0,12.0,26
6,8.0,12.0,16
7,9.0,12.0,3
8,10.0,12.0,51
9,11.0,12.0,3


### Right Shoulder Width change
Node Link-IN  Link-OUT validation: SPEED LIMIT, and NUMBER OF LANES

In [21]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['RS_WIDTH']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'RS_WIDTH':12,
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True
#Export name:
export_name = 'Right_Shoulder_Width'


### RUN FUNCTIONS
network = compare_attributes(attribute_names, attribute_filters, export_geometry, export_name)
network.head()

Total network links with change in RS_WIDTH: 136
Table exported to ..\..\data\Right_Shoulder_Width_RS_WIDTH.csv
Shapefile exported to ..\..\data\Right_Shoulder_Width_RS_WIDTH.shp


,ID,node,in_ID,in_RS_WIDTH,out_RS_WIDTH,diff_RS_WIDTH
0,23177,14964,15719,12.0,0.0,-12.0
1,16852,16112,36882,0.0,12.0,12.0
2,28741,25926,28762,20.0,2.0,-18.0
3,28787,22285,28798,2.0,20.0,18.0
4,16845,16105,37216,0.0,12.0,12.0
...,...,...,...,...,...,...
131,2339,3766,12071,0.0,12.0,12.0
132,78765,2401,78768,12.0,0.0,-12.0
133,78765,2401,78769,12.0,0.0,-12.0
134,78768,2401,38665,0.0,12.0,12.0


### Change in Speed and minor change in Heading
Node Link-IN Link-OUT validation: SPEED LIMIT and HEADING

In [22]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_HEADING']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':25,
}
# Filter by CHANGE in HEADING (Equal to or lesser than)
# Slight change in heading indicates low probability of Turn:
heading_filter = 30
# Summarize by Node Maximum value and Export Geo:
export_geometry = True
#Export name:
export_name = 'Speed_Limit_minorHeading'


### RUN FUNCTIONS
network = compare_attributes(attribute_names, attribute_filters, export_geometry, export_name, heading_filter)
network.head()

Total network links with change in SPD_LMT, _HEADING: 13
Table exported to ..\..\data\Speed_Limit_minorHeading_SPD_LMT__HEADING.csv
Shapefile exported to ..\..\data\Speed_Limit_minorHeading_SPD_LMT.shp


,ID,node,in_ID,in_SPD_LMT,out_SPD_LMT,in__HEADING,out__HEADING,diff_SPD_LMT,diff__HEADING
0,17366,16469,38569,45.0,70.0,58.921408,46.640245,25.0,-12.281162
1,33530,29066,33518,70.0,45.0,301.026323,306.857526,-25.0,5.831203
2,33600,29065,33531,45.0,70.0,289.653367,298.987783,25.0,9.334416
3,17481,16608,38599,45.0,70.0,322.237216,315.104885,25.0,-7.132330
4,12686,12951,15443,55.0,30.0,323.943239,327.116635,-25.0,3.173396


### Change in Number of Lanes and minor change in Heading
Node Link-IN Link-OUT validation: NUMBER OF LANES and HEADING

In [23]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['_LANES', '_HEADING']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    '_LANES':3,
}
# Filter by CHANGE in HEADING (Equal to or lesser than)
# Slight change in heading indicates low probability of Turn:
heading_filter = 30
# Summarize by Node Maximum value and Export Geo:
export_geometry = True
#Export name:
export_name = 'NumberLanes_minorHeading'


### RUN FUNCTIONS
network = compare_attributes(attribute_names, attribute_filters, export_geometry, export_name, heading_filter)
network.head()

Total network links with change in _LANES, _HEADING: 26
Table exported to ..\..\data\NumberLanes_minorHeading__LANES__HEADING.csv
Shapefile exported to ..\..\data\NumberLanes_minorHeading__LANES.shp


,ID,node,in_ID,in__LANES,out__LANES,in__HEADING,out__HEADING,diff__LANES,diff__HEADING
0,15747,14996,15782,4.0,1.0,298.710629,321.162841,-3.0,22.452212
1,15953,15205,16255,4.0,1.0,129.604645,146.713794,-3.0,17.109148
2,15942,15199,78744,1.0,4.0,52.748234,64.339719,3.0,11.591485
3,15751,15002,15775,4.0,1.0,42.756006,54.271635,-3.0,11.515629
4,15917,15102,15944,1.0,4.0,8.507739,30.619352,3.0,22.111613
